## Paramters

In [50]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-5678415b1c0fd33e'
PROJECT = 'qwiklabs-gcp-5678415b1c0fd33e'
REGION = 'us-central1'
SEQ_LEN = 8
N_FORWARD = 5

In [51]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['SEQ_LEN'] = str(SEQ_LEN)
os.environ['N_FORWARD'] = str(N_FORWARD)
os.environ['TFVERSION'] = '1.13'

## Train model locally
Make sure the code works as intended

In [52]:
%%bash
OUTDIR=$(pwd)/trained/linear
rm -rf $OUTDIR

gcloud ml-engine local train \
   --module-name=model.task \
   --package-path=${PWD}/model \
   -- \
   --model=linear \
   --train_data_path="data/train_${SEQ_LEN}_${N_FORWARD}.csv" \
   --eval_data_path="data/eval_${SEQ_LEN}_${N_FORWARD}.csv"  \
   --output_dir=${OUTDIR} \
   --seq_length=$SEQ_LEN \
   --n_forward=$N_FORWARD \
   --train_steps=200 \
   --eval_delay_secs=1 \
   --min_eval_frequency=60


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
2019-07-16 17:44:29.583367: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-07-16 17:44:29.588344: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-07-16 17:44:29.588649: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55b6d1a7e1b0 executing computations on platform Host. Devices:
2019-07-16 17:44:29.588684: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
Instructions for updating:
Use standard file APIs to check for files with this prefix.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf

## Cloud ML Engine
Now to train on Cloud ML Engine with more steps and models.
Copy data to bucket first.

In [16]:
%%bash
gsutil -m rm -rf gs://${BUCKET}/energy/*
gsutil -m cp data/*.csv gs://${BUCKET}/energy

CommandException: 1 files/objects could not be removed.
Copying file://data/eval_8_5.csv [Content-Type=text/csv]...
Copying file://data/train_3_1.csv [Content-Type=text/csv]...
Copying file://data/eval_3_1.csv [Content-Type=text/csv]...
Copying file://data/train_8_5.csv [Content-Type=text/csv]...
/ [4/4 files][870.8 KiB/870.8 KiB] 100% Done                                    
Operation completed over 4 objects/870.8 KiB.                                    


In [53]:
%%bash
for MODEL in linear dnn cnn rnn lstm; do
  OUTDIR=gs://${BUCKET}/project_energy/${MODEL}
  JOBNAME=energy_${MODEL}_$(date -u +%y%m%d_%H%M%S)
  gsutil -m rm -rf $OUTDIR
  gcloud ml-engine jobs submit training $JOBNAME \
     --region=$REGION \
     --module-name=model.task \
     --package-path=${PWD}/model \
     --job-dir=$OUTDIR \
     --scale-tier=BASIC \
     --runtime-version=$TFVERSION \
     -- \
     --model=$MODEL \
     --train_data_path="gs://${BUCKET}/energy/train_${SEQ_LEN}_${N_FORWARD}.csv" \
     --eval_data_path="gs://${BUCKET}/energy/eval_${SEQ_LEN}_${N_FORWARD}.csv"  \
     --output_dir=${OUTDIR} \
     --seq_length=$SEQ_LEN \
     --n_forward=$N_FORWARD \
     --train_steps=5000 \
     --eval_delay_secs=1 \
     --min_eval_frequency=100
done

jobId: energy_linear_190716_174511
state: QUEUED
jobId: energy_dnn_190716_174514
state: QUEUED
jobId: energy_cnn_190716_174518
state: QUEUED
jobId: energy_rnn_190716_174521
state: QUEUED
jobId: energy_lstm_190716_174525
state: QUEUED


Removing gs://qwiklabs-gcp-5678415b1c0fd33e/project_energy/linear/packages/524f4b6241ed0ae19859c238716598eb722304e71f05b09cdf5d5d51a8bc7d75/model-0.0.0.tar.gz#1563299006713489...
/ [1/1 objects] 100% Done                                                       
Operation completed over 1 objects.                                              
Job [energy_linear_190716_174511] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe energy_linear_190716_174511

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs energy_linear_190716_174511
Removing gs://qwiklabs-gcp-5678415b1c0fd33e/project_energy/dnn/packages/864f99d3f3b02e207910f748f3ec2c493a13b23594514207b63a3d51ef60f7e3/model-0.0.0.tar.gz#1563299010392589...
/ [1/1 objects] 100% Done                                                       
Operation completed over 1 objects.                                             